In [3]:
%run utils.ipynb

In [ ]:
def check(row):
    global TRADES, trade_in_progress, signals, holding_period, check_trend_change
    
    if(trade_in_progress):
        _data = TRADES[-1]
        # time to sell after n holding days
        if(row['index']==_data['sell_index']):
            _data['sell_price'] = row['Adj Close']
            _data['sell_date'] = str(pd.to_datetime(row['Date']).date())
            _data['returns'] = round((_data['sell_price']-_data['buy_price'])/_data['buy_price']*100, 2)
            TRADES[-1] = _data
            trade_in_progress = False
                
    else:
        _r = row.merge(signals, how='inner', on=[signals.columns[:-1]])
        strategy = _r.shape[0]
        
        if(strategy>0): 
            trade_in_progress = True
            _data = {
                'grade': _r['signal_grade'].values[0],
                'buy_date': str(pd.to_datetime(row['Date']).date()),
                'buy_index': row['index'],
                'sell_date': '',
                'sell_index': row['index'] + holding_period,
                'buy_price': row['Adj Close'],
                'sell_price': '',
                'returns': 0           
            }
            TRADES.append(_data)
            del _data
